<a href="https://colab.research.google.com/github/27luiz/dadosSUS/blob/main/Dadosus_AIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import auth
auth.authenticate_user()


In [8]:
from ftplib import FTP
from google.cloud import storage
import threading
from datetime import datetime, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

def download_and_upload(date, gcs_bucket_name):
    ano = date.strftime("%y")
    mes = date.strftime("%m")
    uf = 'DF'  # Aqui você pode definir a UF desejada

    ftp_url = ftp_base_url.format(uf=uf, ano=ano, mes=mes)
    local_filename = f"RD{uf}{ano}{mes}.dbc"
    gcs_blob_name = f"bruto/RD{uf}{ano}{mes}.dbc"

    try:
        # Download do arquivo FTP
        download_ftp_file(ftp_url, local_filename)

        # Upload do arquivo para o Google Cloud Storage
        upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name)
    except Exception as e:
        print(f"Erro ao processar UF={uf}, Ano={ano}, Mês={mes}: {e}")

def download_ftp_file(ftp_url, local_filename):
    ftp_server = ftp_url.split('/')[2]
    ftp_path = "/".join(ftp_url.split('/')[3:])

    with FTP(ftp_server) as ftp:
        ftp.login()
        with open(local_filename, 'wb') as f:
            ftp.retrbinary(f'RETR {ftp_path}', f.write)

def upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket_name)
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)
    print(f"Arquivo enviado com sucesso para o Google Cloud Storage: {gcs_blob_name}")

# URL base do arquivo FTP
ftp_base_url = "ftp://ftp.datasus.gov.br/dissemin/publicos/SIHSUS/200801_/Dados/RD{uf}{ano}{mes}.dbc"

# Localização no Google Cloud Storage para onde o arquivo será carregado
gcs_bucket_name = "tabsus"

# Intervalo de datas
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 1)

# Lista para armazenar as threads
threads = []

# Criar e iniciar threads para cada data no intervalo
for single_date in daterange(start_date, end_date):
    thread = threading.Thread(target=download_and_upload, args=(single_date, gcs_bucket_name))
    thread.start()
    threads.append(thread)

# Aguardar todas as threads terminarem
for thread in threads:
    thread.join()

print("Todos os arquivos foram enviados com sucesso.")


Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDDF2401.dbc
Todos os arquivos foram enviados com sucesso.
